In [ ]:
!which python3

In [ ]:
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, models, transforms
import time
import copy
from PIL import Image

In [ ]:
from collections import OrderedDict

In [ ]:
import torchvision.transforms.functional as TF
from runpy import run_path
from skimage import img_as_ubyte
from natsort import natsorted
from glob import glob

In [ ]:
import cv2
import torchinfo
import numpy as np
from torch.optim import Adam
from torch.utils.data import DataLoader

In [ ]:
import io
import imageio
from tqdm import tqdm
from avi_r import AVIReader
import matplotlib.pyplot as plt
from ipywidgets import widgets, HBox

In [ ]:
import os
import torch
import torch.nn as nn

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])

In [ ]:
device

In [ ]:
WalkingDataset = np.load('train_on_beach.npy')
# WalkingDataset = np.load('train_in_house.npy')
print(WalkingDataset.shape)

In [ ]:
np.random.shuffle(WalkingDataset)

In [ ]:
dataset = WalkingDataset[:,5:100]

In [ ]:
class ConvLSTMCell(nn.Module):

    def __init__(self, in_channels, out_channels, 
    kernel_size, padding, activation, frame_size):

        super(ConvLSTMCell, self).__init__()  

        if activation == "tanh":
            self.activation = torch.tanh 
        elif activation == "relu":
            self.activation = torch.relu
        
        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        self.conv = nn.Conv2d(
            in_channels=in_channels + out_channels, 
            out_channels=4 * out_channels, 
            kernel_size=kernel_size, 
            padding=padding)           

        # Initialize weights for Hadamard Products
        self.W_ci = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_co = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_cf = nn.Parameter(torch.Tensor(out_channels, *frame_size))

    def forward(self, X, H_prev, C_prev):

        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        conv_output = self.conv(torch.cat([X, H_prev], dim=1))

        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        i_conv, f_conv, C_conv, o_conv = torch.chunk(conv_output, chunks=4, dim=1)

        input_gate = torch.sigmoid(i_conv + self.W_ci * C_prev )
        forget_gate = torch.sigmoid(f_conv + self.W_cf * C_prev )

        # Current Cell output
        C = forget_gate*C_prev + input_gate * self.activation(C_conv)

        output_gate = torch.sigmoid(o_conv + self.W_co * C )

        # Current Hidden State
        H = output_gate * self.activation(C)

        return H, C

In [ ]:
class ConvLSTM(nn.Module):

    def __init__(self, in_channels, out_channels, 
    kernel_size, padding, activation, frame_size):

        super(ConvLSTM, self).__init__()

        self.out_channels = out_channels

        # We will unroll this over time steps
        self.convLSTMcell = ConvLSTMCell(in_channels, out_channels, 
        kernel_size, padding, activation, frame_size)

    def forward(self, X):

        # X is a frame sequence (batch_size, num_channels, seq_len, height, width)

        # Get the dimensions
        batch_size, _, seq_len, height, width = X.size()

        # Initialize output
        output = torch.zeros(batch_size, self.out_channels, seq_len, 
        height, width, device=device)
        
        # Initialize Hidden State
        H = torch.zeros(batch_size, self.out_channels, 
        height, width, device=device)

        # Initialize Cell Input
        C = torch.zeros(batch_size,self.out_channels, 
        height, width, device=device)

        # Unroll over time steps
        for time_step in range(seq_len):

            H, C = self.convLSTMcell(X[:,:,time_step], H, C)

            output[:,:,time_step] = H

        return output

In [ ]:
class Seq2Seq(nn.Module):

    def __init__(self, num_channels, num_kernels, kernel_size, padding, 
    activation, frame_size, num_layers):

        super(Seq2Seq, self).__init__()

        self.sequential = nn.Sequential()

        # Add First layer (Different in_channels than the rest)
        self.sequential.add_module(
            "convlstm1", ConvLSTM(
                in_channels=num_channels, out_channels=num_kernels,
                kernel_size=kernel_size, padding=padding, 
                activation=activation, frame_size=frame_size)
        )

        self.sequential.add_module(
            "batchnorm1", nn.BatchNorm3d(num_features=num_kernels)
        ) 

        # Add rest of the layers
        for l in range(2, num_layers+1):

            self.sequential.add_module(
                f"convlstm{l}", ConvLSTM(
                    in_channels=num_kernels, out_channels=num_kernels,
                    kernel_size=kernel_size, padding=padding, 
                    activation=activation, frame_size=frame_size)
                )
                
            self.sequential.add_module(
                f"batchnorm{l}", nn.BatchNorm3d(num_features=num_kernels)
                ) 

        # Add Convolutional Layer to predict output frame
        self.conv = nn.Conv2d(
            in_channels=num_kernels, out_channels=num_channels,
            kernel_size=kernel_size, padding=padding)

    def forward(self, X):

        # Forward propagation through all the layers
        output = self.sequential(X)

        # Return only the last output frame
        output = self.conv(output[:,:,-1])
        
        return nn.Sigmoid()(output)

In [ ]:
# The input video frames are grayscale, thus single channel
model_video_frame_pred = Seq2Seq(num_channels = 1, num_kernels = 64, 
                kernel_size = (3, 3), padding = (1, 1), activation="relu", 
                frame_size = (120, 160), num_layers = 3).to(device)

optim = Adam(model_video_frame_pred.parameters(), lr=1e-4)

# Binary Cross Entropy, target pixel values either 0 or 1
criterion = nn.BCELoss(reduction = 'sum')
model_video_frame_pred.load_state_dict(torch.load("/home/staditya/Desktop/Pushkal/IVP Project/Proposed Model/tmp.pth"))

In [ ]:
%cd Restormer
# task = 'Real_Denoising'
task = 'Motion_Deblurring'

def get_weights_and_parameters(task, parameters):
    if task == 'Motion_Deblurring':
        weights = os.path.join('Motion_Deblurring', 'pretrained_models', 'motion_deblurring.pth')
    elif task == 'Single_Image_Defocus_Deblurring':
        weights = os.path.join('Defocus_Deblurring', 'pretrained_models', 'single_image_defocus_deblurring.pth')
    elif task == 'Deraining':
        weights = os.path.join('Deraining', 'pretrained_models', 'deraining.pth')
    elif task == 'Real_Denoising':
        weights = os.path.join('Denoising', 'pretrained_models', 'real_denoising.pth')
        parameters['LayerNorm_type'] = 'BiasFree'
    return weights, parameters


# Get model weights and parameters
parameters = {'inp_channels':3, 'out_channels':3, 'dim':48, 'num_blocks':[4,6,6,8], 'num_refinement_blocks':4, 'heads':[1,2,4,8], 'ffn_expansion_factor':2.66, 'bias':False, 'LayerNorm_type':'WithBias', 'dual_pixel_task':False}
weights, parameters = get_weights_and_parameters(task, parameters)

load_arch = run_path(os.path.join('basicsr', 'models', 'archs', 'restormer_arch.py'))
model_deblur = load_arch['Restormer'](**parameters)
model_deblur.to(device) # model.cuda()

checkpoint = torch.load(weights)
model_deblur.load_state_dict(checkpoint['params'])
%cd ..

In [ ]:
%cd Restormer
task = 'Real_Denoising'
# task = 'Motion_Deblurring'

def get_weights_and_parameters(task, parameters):
    if task == 'Motion_Deblurring':
        weights = os.path.join('Motion_Deblurring', 'pretrained_models', 'motion_deblurring.pth')
    elif task == 'Single_Image_Defocus_Deblurring':
        weights = os.path.join('Defocus_Deblurring', 'pretrained_models', 'single_image_defocus_deblurring.pth')
    elif task == 'Deraining':
        weights = os.path.join('Deraining', 'pretrained_models', 'deraining.pth')
    elif task == 'Real_Denoising':
        weights = os.path.join('Denoising', 'pretrained_models', 'real_denoising.pth')
        parameters['LayerNorm_type'] = 'BiasFree'
    return weights, parameters


# Get model weights and parameters
parameters = {'inp_channels':3, 'out_channels':3, 'dim':48, 'num_blocks':[4,6,6,8], 'num_refinement_blocks':4, 'heads':[1,2,4,8], 'ffn_expansion_factor':2.66, 'bias':False, 'LayerNorm_type':'WithBias', 'dual_pixel_task':False}
weights, parameters = get_weights_and_parameters(task, parameters)

load_arch = run_path(os.path.join('basicsr', 'models', 'archs', 'restormer_arch.py'))
model_denoise = load_arch['Restormer'](**parameters)
model_denoise.to(device) # model.cuda()

checkpoint = torch.load(weights)
model_denoise.load_state_dict(torch.load("/home/staditya/Desktop/Pushkal/IVP Project/Proposed Model/denoising.pth"))
%cd ..

In [ ]:
def collate_test(batch):

    # 45th to 60th frames are target
    target = np.array(batch)[:,-1]                     
    
    # Add channel dim, scale pixels between 0 and 1, send to GPU
    batch = torch.tensor(batch, dtype = torch.float32).unsqueeze(1)          
    batch = batch / 255.0                          
    return batch[:,:,:40].to(device), target

In [ ]:
final_gt = []
final_output = []

for data_i in range(20):
    data = dataset[data_i]
    outputs = []
    ground_truths = []

    for i in range(8):
        previous_block = dataset[data_i, i : 40]

        if i > 0:
            new_block = np.concatenate((previous_block, np.array(outputs).squeeze(1)), axis = 0)
        else:
            new_block = previous_block

        target = dataset[data_i, 40 + i]
        ground_truths.append(target)

        new_block = np.concatenate((new_block, [target]), axis = 0)

        test_loader = DataLoader([new_block], batch_size=1, collate_fn=collate_test)
        batch, tgt = next(iter(test_loader))

        inp = batch[:,:, : 40]
        output_vfp = (model_video_frame_pred(inp).squeeze(1).cpu().detach().numpy().astype(float)) * 255.0
        
        #############################################
        
        # Converting image to 3 channel from one channel
        im = Image.fromarray(output_vfp[0]).convert("L")
        im.save("tmp.png")
        im = cv2.cvtColor(cv2.imread("tmp.png"), cv2.COLOR_BGR2RGB)
        inp_to_deblur = torch.from_numpy(im).float().div(255.).permute(2,0,1).unsqueeze(0).to(device)
        
        output_1 = model_deblur(inp_to_deblur)
        output = (model_denoise(output_1).squeeze().cpu().detach().numpy().astype(float)) * 255.0
        
        
        # Converting image from 3 channel to one channel
        output = rgb2gray(np.swapaxes(np.swapaxes(output, 0, 2), 0, 1))
        
        #############################################
        
        outputs.append([output])
        
    fps = 5
    tmp = data_i + 1
    ground_truths = np.array(ground_truths, dtype = np.uint8)
    outputs = np.array(outputs, dtype = np.uint8).squeeze()

    with io.BytesIO() as gif:
        imageio.mimsave(gif, ground_truths, "GIF", fps = fps)    
        output_gif = gif.getvalue()

    with io.BytesIO() as gif:
        imageio.mimsave(gif, outputs, "GIF", fps = fps)    
        target_gif = gif.getvalue()

    print(f"\nTest video: {tmp}")
    display(HBox([widgets.Image(value=output_gif), 
                  widgets.Image(value=target_gif)]))
    
    final_gt.append(ground_truths)
    final_output.append(outputs)